In [ ]:
#| default_exp flakes

In [ ]:
# greet('dutil')

In [ ]:
import sys
if '/app/data' not in sys.path: sys.path.append('/app/data')

**NOTE**: the import block below should be tagged for export, but is left unexported for demonstration purposes.

In [ ]:
#| export
from collections import defaultdict
from itertools import repeat
from pathlib import Path
import re
from urllib.parse import quote_plus
import fastcore.all as FC
import pyflakes
from pyflakes.reporter import Reporter
from pyflakes.messages import Message
from pyflakes.api import check as _check
from dialoghelper.core import find_msgs, find_dname, find_msg_id, find_var, read_msg, update_msg, add_msg

In [ ]:
from functools import reduce
from IPython.display import HTML, Markdown
from itertools import product, accumulate, repeat

import nbdev.config
import fastcore.all as FC
from rich.console import Console
from fastcore.test import *
from fasthtml.components import Button
from dialoghelper.stdtools import *


In [ ]:
cprint = Console(width=140, tab_size=4, force_jupyter=True if FC.IN_IPYTHON else None).print

## setup - install pyflakes

**NOTE**: not cool, no one should be obligued to install a package just for perusing a notebook. There's nothing I can do for now -> see exploration [uv run & PEP 876](/dialog_?name=vic/explorer/uv)

`pyflakes` is a very lightweight, zero dependencies, python package.

In [ ]:
!pip install -U pyflakes

### download companion dialog

**NOTE**: if you want to follow along with the same file I used to test this dialog, download it from [dutil/nbs/00_core.ipynb](https://raw.githubusercontent.com/civvic/dutil/refs/heads/main/nbs/00_core.ipynb) and put it side by side this dialog. Otherwise, edit `_DREL` below to point to some other local dialog (must be relative to where this dialog is, no starting with '/'.)

In [ ]:
_DREL = '00_core'
_DABS = f"/{Path(find_dname()).parent}/{_DREL}"

assert (Path.home()/_DABS[1:]).with_suffix('.ipynb').exists()
_DREL, _DABS

('00_core', '/vic/dutil/nbs/00_core')

# flakes

I'm working on **dutil**, a nbdev project. I want to create a utility function that checks for import issues in exported code cells before running `nbdev_export()`.

The problem: When developing in a notebook/dialog, code can work fine because symbols are available in the global scope, but when exported to a module, missing imports cause failures. This is especially problematic inside function bodies where a symbol might not be imported.

Solution approach: Use `pyflakes` to check the concatenated exported code cells and report undefined names. This gives us a lightweight way to catch import (and many other) issues without running a full lint/type checker.

We'll need to:
1. Extract all exported code cells from the current dialog
2. Concatenate them into a string
3. Run pyflakes on it
4. Return structured warnings

## API usage

In [ ]:
cds = find_msgs(msg_type='code')
len(cds)

83

In [ ]:
src = '\n'.join(cds.filter(lambda x: x.is_exported).itemgot('content'))
print(src[:500])

@FC.patch
def as_tuple(self: Message): return type(self).__name__, self.filename, (self.lineno, self.col+1), self.message %self.message_args, self.message_args

class ListReporter(Reporter):
    def __init__(self): self.warnings = FC.L(); super().__init__(None, None)
    def flake(self, message): self.warnings.append(message.as_tuple())
def _line2msgid(codes, msgids):
    line, l2id = 1, FC.L()
    for code, msgid in zip(codes, msgids):
        code = f"# %% msgid: {msgid}\n{code}\n".split('\n')


In [ ]:
%%capture captured_output
srcname = '05_flakes.py'
_check(src, srcname)

In [ ]:
print(captured_output.stdout[:200])

05_flakes.py:1:2: undefined name 'FC'
05_flakes.py:2:20: undefined name 'Message'
05_flakes.py:4:20: undefined name 'Reporter'
05_flakes.py:5:41: undefined name 'FC'
05_flakes.py:8:21: undefined name 


In [ ]:

fn_rng, warn = "05_flakes.py:2:2: undefined name 'FC'".split(': ')
fn, _, rng = fn_rng.partition(':')
fn, rng, warn

('05_flakes.py', '2:2', "undefined name 'FC'")

## custom reporter

In [ ]:
#| export
@FC.patch
def as_tuple(self: Message): return type(self).__name__, self.filename, (self.lineno, self.col+1), self.message %self.message_args, self.message_args

class ListReporter(Reporter):
    def __init__(self): self.warnings = FC.L(); super().__init__(None, None)
    def flake(self, message): self.warnings.append(message.as_tuple())

In [ ]:
_check(src, srcname, reprt := ListReporter())

23

In [ ]:
for _ in reprt.warnings[:5]: print(_)

('UndefinedName', '05_flakes.py', (1, 2), "undefined name 'FC'", ('FC',))
('UndefinedName', '05_flakes.py', (2, 20), "undefined name 'Message'", ('Message',))
('UndefinedName', '05_flakes.py', (4, 20), "undefined name 'Reporter'", ('Reporter',))
('UndefinedName', '05_flakes.py', (5, 41), "undefined name 'FC'", ('FC',))
('UndefinedName', '05_flakes.py', (8, 21), "undefined name 'FC'", ('FC',))


## v1

In [ ]:
def check_flakes(src:str=None, srcname:str=None):
    if not src:
        cds = find_msgs(msg_type='code')
        src = '\n'.join(cds.filter(lambda x: x.is_exported).itemgot('content'))
    if not srcname:
        srcname = (Path.home()/find_dname()).with_suffix('.py').name
    _check(src, srcname, reprt := ListReporter())
    return reprt.warnings
    

In [ ]:
for wrn in check_flakes(src, srcname)[:5]: print(wrn)

('UndefinedName', '05_flakes.py', (1, 2), "undefined name 'FC'", ('FC',))
('UndefinedName', '05_flakes.py', (2, 20), "undefined name 'Message'", ('Message',))
('UndefinedName', '05_flakes.py', (4, 20), "undefined name 'Reporter'", ('Reporter',))
('UndefinedName', '05_flakes.py', (5, 41), "undefined name 'FC'", ('FC',))
('UndefinedName', '05_flakes.py', (8, 21), "undefined name 'FC'", ('FC',))


In [ ]:
for wrn in check_flakes()[:5]: print(wrn)

('UndefinedName', '05_flakes.py', (1, 2), "undefined name 'FC'", ('FC',))
('UndefinedName', '05_flakes.py', (2, 20), "undefined name 'Message'", ('Message',))
('UndefinedName', '05_flakes.py', (4, 20), "undefined name 'Reporter'", ('Reporter',))
('UndefinedName', '05_flakes.py', (5, 41), "undefined name 'FC'", ('FC',))
('UndefinedName', '05_flakes.py', (8, 21), "undefined name 'FC'", ('FC',))


## v2

In [ ]:
nbdev.config.is_nbdev(), nbdev.config.get_config().lib_path


(True, Path('/app/data/vic/dutil/dutil'))

In [ ]:
('#| hide'.strip()[2:]).strip().split()

['hide']

In [ ]:
def check_flakes(
    src:str=None,
    srcname:str=None, 
):
    "Check for pyflakes 'warnings' in `src` or current dialog"
    if not src:
        cds = find_msgs(msg_type='code')
        src = '\n'.join(cds.filter(lambda x: x.is_exported).itemgot('content'))
        if not srcname and nbdev.config.is_nbdev():
            if s := FC.first(m.content for m in cds if m.content and re.match(r'\s*#\s*\|', m.content.split()[0])):
                dirct = (s.strip()[2:]).strip().split()
                if dirct[0] == 'default_exp' and len(dirct) > 1: srcname = f"{dirct[1]}.py"
    if not srcname:
        srcname = (Path.home()/find_dname()).with_suffix('.py').name
    _check(src, srcname, reprt := ListReporter())
    return reprt.warnings
    

In [ ]:
for wrn in check_flakes()[:5]: print(wrn)

('UndefinedName', 'flakes.py', (1, 2), "undefined name 'FC'", ('FC',))
('UndefinedName', 'flakes.py', (2, 20), "undefined name 'Message'", ('Message',))
('UndefinedName', 'flakes.py', (4, 20), "undefined name 'Reporter'", ('Reporter',))
('UndefinedName', 'flakes.py', (5, 41), "undefined name 'FC'", ('FC',))
('UndefinedName', 'flakes.py', (8, 21), "undefined name 'FC'", ('FC',))


## line 2 msgid

In [ ]:
def _code_with_msgid(codes, msgids):
    "Concatenate code blocks with msgids and track line ranges"
    s, line, l2id = '', 1, {}
    for code, msgid in zip(codes, msgids):
        s += f"{'\n' if line > 1 else ''}# %% msgid: {msgid}\n" + code
        end = line + code.count('\n') + 1
        l2id |= dict(product(range(line, end), (msgid,)))
        line = end
    l2id |= dict(product(range(line, len(s.splitlines())+1), (msgid,)))
    return s, l2id


In [ ]:
codes = ['import os\n\nprint("hi")', 'def foo():\n    return 42', 'x = foo()\nprint(x)']
msgids = ['_abc123', '_def456', '_ghi789']

code, l2id = _code_with_msgid(codes, msgids)
cprint('\n'.join(f"{i+1:3} {line}" for i,line in enumerate(code.split('\n'))))
cprint('\n',l2id)

1 # %% msgid: _abc123
  2 import os
  3 
  4 print("hi")
  5 # %% msgid: _def456
  6 def foo():
  7     return 42
  8 # %% msgid: _ghi789
  9 x = foo()
 10 print(x)

{
    1: '_abc123',
    2: '_abc123',
    3: '_abc123',
    4: '_def456',
    5: '_def456',
    6: '_ghi789',
    7: '_ghi789',
    8: '_ghi789',
    9: '_ghi789',
    10: '_ghi789'
}

In [ ]:
try:
    test_eq(l2id[3], '_abc123')
    test_eq(l2id[5], '_abc123')
    test_eq(l2id[6], '_def456')
    test_eq(l2id[10], '_ghi789')
except Exception as e:
    display(e)

AssertionError('==:\n_def456\n_abc123')

In [ ]:
s = '\n\n'.join(f"# %% msgid: {msgid}\n{code}" for code, msgid in zip(codes, msgids))
cprint('\n'.join(f"{i+1:3} {line}" for i,line in enumerate(s.split('\n'))))

1 # %% msgid: _abc123
  2 import os
  3 
  4 print("hi")
  5 
  6 # %% msgid: _def456
  7 def foo():
  8     return 42
  9 
 10 # %% msgid: _ghi789
 11 x = foo()
 12 print(x)

In [ ]:
line = 1
code, msgid = codes[0], msgids[0]
end = line + code.count('\n') + 3  # base 1, + 2 cr
end

6

In [ ]:
line = 6
code, msgid = codes[1], msgids[1]
end = line + code.count('\n') + 3  # base 1, + 2 cr
end

10

In [ ]:
line = 10
code, msgid = codes[2], msgids[2]
end = min(s.count('\n')+1, line + code.count('\n') + 3)  # base 1, + 2 cr
end

12

In [ ]:
line, ls, l2id = 1, s.count('\n')+1, {}
for code, msgid in zip(codes, msgids):
    end = min(ls+1, line + code.count('\n') + 3)
    l2id |= {l:m for l,m in product(range(line, end), ((msgid, line),))}
    line = end
print(l2id)


{1: ('_abc123', 1), 2: ('_abc123', 1), 3: ('_abc123', 1), 4: ('_abc123', 1), 5: ('_abc123', 1), 6: ('_def456', 6), 7: ('_def456', 6), 8: ('_def456', 6), 9: ('_def456', 6), 10: ('_ghi789', 10), 11: ('_ghi789', 10), 12: ('_ghi789', 10)}


In [ ]:
cprint('\n'.join(f"{i+1:3} {line}" for i,line in enumerate(s.split('\n'))))

1 # %% msgid: _abc123
  2 import os
  3 
  4 print("hi")
  5 
  6 # %% msgid: _def456
  7 def foo():
  8     return 42
  9 
 10 # %% msgid: _ghi789
 11 x = foo()
 12 print(x)

In [ ]:
line, ls, l2id = 1, s.count('\n')+1, {}
for code, msgid in zip(codes, msgids):
    end = min(ls+1, line + code.count('\n') + 3)
    l2id |= {l:m for l,m in zip(range(line, end), product(range(1, len(code)+2), (msgid,)))}
    line = end
print(l2id)


{1: (1, '_abc123'), 2: (2, '_abc123'), 3: (3, '_abc123'), 4: (4, '_abc123'), 5: (5, '_abc123'), 6: (1, '_def456'), 7: (2, '_def456'), 8: (3, '_def456'), 9: (4, '_def456'), 10: (1, '_ghi789'), 11: (2, '_ghi789'), 12: (3, '_ghi789')}


In [ ]:
#| export
def _line2msgid(codes, msgids):
    line, l2id = 1, FC.L()
    for code, msgid in zip(codes, msgids):
        code = f"# %% msgid: {msgid}\n{code}\n".split('\n')
        end = line + code.count('\n') + 2
        l2id.extend(zip(code, range(0, len(code)-1), repeat(msgid)))
        line = end
    return l2id

In [ ]:
l2id = _line2msgid(codes, msgids)
print('\n'.join(f"{i+1:2}: {l[1]} {l[2]} - {l[0]}" for i, l in l2id.enumerate()))
src = '\n'.join(l2id.itemgot(0))
print(src)

 1: 0 _abc123 - # %% msgid: _abc123
 2: 1 _abc123 - import os
 3: 2 _abc123 - 
 4: 3 _abc123 - print("hi")
 5: 0 _def456 - # %% msgid: _def456
 6: 1 _def456 - def foo():
 7: 2 _def456 -     return 42
 8: 0 _ghi789 - # %% msgid: _ghi789
 9: 1 _ghi789 - x = foo()
10: 2 _ghi789 - print(x)
# %% msgid: _abc123
import os

print("hi")
# %% msgid: _def456
def foo():
    return 42
# %% msgid: _ghi789
x = foo()
print(x)


## code messages

In [ ]:
Path(find_dname()).name


'05_flakes'

In [ ]:
(Path.home()/find_dname()).with_suffix('.ipynb')

Path('/app/data/vic/dutil/nbs/05_flakes.ipynb')

In [ ]:
dname = _DABS

dname

'/vic/dutil/nbs/00_core'

In [ ]:
cds = find_msgs(msg_type='code', dname=dname)
exptd = cds.filter(lambda x: x.is_exported)
for _ in exptd.attrgot('content'): cprint(_[:100])

import re
import sys
import inspect
from typing import Any
import fastcore.all as FC
from fastcore.x

def solveit_version():
    "Return the version of solveit if it is found"
    s = ' '.join(_.__modul

def in_dialog():
    "Check if the code is running in a solveit dialog"
    return bool(solveit_vers

def get_caller_globals(): 
    "Return the globals of the caller"
    return inspect.currentframe().

_empty = inspect.Parameter.empty

def at_(
    o, # Object to traverse (dict, list, object, or nested combination)
    sym: str, # Pat

def get_tool_names(ns=None, exclude=None, only_exported=False, exclude_private=True):
    "Return th

def add_tools_card(ns=None):
    "Add a message with all tools in namespace `ns` or caller globals"

def setup_ns(ns=None, **kwargs):
    "Add `kwargs` to the namespace `ns` or current dialog"
    ns =

In [ ]:
len(cds), len(exptd)

(79, 9)

In [ ]:
#| export
def _get_source(msgs):
    l2id = _line2msgid(msgs.attrgot('content'), msgs.attrgot('id'))
    return '\n'.join(l2id.itemgot(0)), l2id

In [ ]:
src, l2id = _get_source(exptd)

cprint(src[:400])

# %% msgid: _e72f67fd
import re
import sys
import inspect
from typing import Any
import fastcore.all as FC
from fastcore.xtras import is_listy
import dialoghelper
from dialoghelper import *
from dialoghelper.core import _find_frame_dict
# %% msgid: _6389d58d
def solveit_version():
    "Return the version of solveit if it is found"
    s = ' '.join(_.__module__ for _ in sys.meta_path)
    mtch = re

In [ ]:
src, l2id = _get_source(find_msgs(msg_type='code').filter(lambda x: x.is_exported))
cprint(src)

# %% msgid: _42c1d335
@FC.patch
def as_tuple(self: Message): return type(self).__name__, self.filename, (self.lineno, self.col+1), self.message %self.message_args, 
self.message_args

class ListReporter(Reporter):
    def __init__(self): self.warnings = FC.L(); super().__init__(None, None)
    def flake(self, message): self.warnings.append(message.as_tuple())
# %% msgid: _28266a58
def _line2msgid(codes, msgids):
    line, l2id = 1, FC.L()
    for code, msgid in zip(codes, msgids):
        code = f"# %% msgid: {msgid}\n{code}\n".split('\n')
        end = line + code.count('\n') + 2
        l2id.extend(zip(code, range(0, len(code)-1), repeat(msgid)))
        line = end
    return l2id
# %% msgid: _082b8d5b
def _get_source(msgs):
    l2id = _line2msgid(msgs.attrgot('content'), msgs.attrgot('id'))
    return '\n'.join(l2id.itemgot(0)), l2id
# %% msgid: _5e564084
def _warningtypes():
    "Return all pyflakes message types"
    pfmod = pyflakes.messages
    kls_msg = pfmod.Message
    syms = [_ for _ in dir(pfmod) if _[0] != '_']
    return {sym: o.__doc__.strip() if o.__doc__ else o.message
        for sym in syms 
        if sym != 'Message' and isinstance(o := getattr(pfmod, sym), type) and kls_msg in o.mro()}
# %% msgid: _a3a33a79
_ALL = ','.join(_warningtypes().keys())
_IMPORTS = 'ImportShadowedByLoopVar,ImportStarNotPermitted,ImportStarUsage,ImportStarUsed,LateFutureImport,UnusedImport'
_VARS = 'UndefinedExport,UndefinedLocal,UndefinedName,UnusedIndirectAssignment,UnusedVariable'

# %% msgid: _65ee988a
def check_flakes(dname:str='', wtypes:str=_ALL):
    "Check for pyflakes 'warnings' in `sname` or current dialog"
    cds = find_msgs(msg_type='code', dname=dname)
    src, l2id = _get_source(cds.filter(lambda x: x.is_exported))
    nw = _check(src, Path(dname or find_dname()).name, reprt := ListReporter())
    if wtypes != _ALL: wtypes = wtypes.split(',')
    return nw, l2id, reprt.warnings if wtypes == _ALL else reprt.warnings.filter(lambda x: x[0] in wtypes)
# %% msgid: _252cd459
def _group_flakes(wns, l2id):
    "Group pyflakes warnings by msgid and warning type"
    res = defaultdict(lambda:defaultdict(list))
    for wtype, _, rng, *rest in wns:
        msgid = l2id[rng[0]][2]
        res.append((rng, rest))
    return res
# %% msgid: _634b288b
def _tag(name, msgid=''): return f"<!-- {name}: {msgid or find_msg_id()} -->"
# %% msgid: _47ffa603
_lnks = (
    '''<span hx-on-click="setTimeout(() => selectMsg($('%s'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer 
hover:bg-muted truncate">**%s**</span>  \n\n''',
    '''<h5 class="uk-flex"><a class="uk-link" href="%s"><strong>%s</strong></svg></a>&nbsp;<svg xmlns="http://www.w3.org/2000/svg" 
viewBox="0 0 24 24" height="16px" width="16px" class="lucide-icon "><use href="#lc-external-link"></use></h5>  \n\n'''
)
# %% msgid: _9513dbdc
def _render_flakes_report(dname, wns, l2id):
    tag = _tag('flakes report')
    dlnk, s = '', f"{tag}\n"
    if dname:
        q = quote_plus(str(Path(find_dname()).parent/dname) if (not dname or dname[0] != '/') else dname[1:])
        dlnk = f"/dialog_?name={q}"
    grouped = _group_flakes(wns, l2id)
    for msgid, wtypes in grouped.items():
        lnk = f"{dlnk}#{msgid}"
        s += _lnks % (lnk, msgid)
        for wtype, ws in wtypes.items():
            s += f"- **{wtype}**: {len(ws)}  \n"
            for w in ws: s += f"    `{w[0]}`: {w[1][0].replace("'", "`")}  \n"
        s += '\n'
    return s, tag
# %% msgid: _dfb18b57
get_ipython().xpush(__linked_msgs={})  # WARNING: neither get_ipython (in user_ns) nor ipykernel_helper.xpush are documented
# %% msgid: _5084f06f
def _update_linked_msg(content, tag, msgid=''):
    msgid = msgid or find_msg_id()
    linked = find_var('__linked_msgs')
    if reportid := linked.get(msgid):
        msg = read_msg(0, True, reportid, nums=True)
        if 'msg' not in msg and re.search(tag, msg.content):
            linked = update_msg(reportid, content=content)
            return
   

## warning types

In [ ]:
pyflakes.messages??


```python
"""
Provide the class Message and its subclasses.
"""


class Message:
    message = ''
    message_args = ()

    def __init__(self, filename, loc):
        self.filename = filename
        self.lineno = loc.lineno
        self.col = loc.col_offset

    def __str__(self):
        return '{}:{}:{}: {}'.format(self.filename, self.lineno, self.col+1,
                                     self.message % self.message_args)


class UnusedImport(Message):
    message = '%r imported but unused'

    def __init__(self, filename, loc, name):
        Message.__init__(self, filename, loc)
        self.message_args = (name,)


class RedefinedWhileUnused(Message):
    message = 'redefinition of unused %r from line %r'

    def __init__(self, filename, loc, name, orig_loc):
        Message.__init__(self, filename, loc)
        self.message_args = (name, orig_loc.lineno)


class ImportShadowedByLoopVar(Message):
    message = 'import %r from line %r shadowed by loop variable'

    def __init__(self, filename, loc, name, orig_loc):
        Message.__init__(self, filename, loc)
        self.message_args = (name, orig_loc.lineno)


class ImportStarNotPermitted(Message):
    message = "'from %s import *' only allowed at module level"

    def __init__(self, filename, loc, modname):
        Message.__init__(self, filename, loc)
        self.message_args = (modname,)


class ImportStarUsed(Message):
    message = "'from %s import *' used; unable to detect undefined names"

    def __init__(self, filename, loc, modname):
        Message.__init__(self, filename, loc)
        self.message_args = (modname,)


class ImportStarUsage(Message):
    message = "%r may be undefined, or defined from star imports: %s"

    def __init__(self, filename, loc, name, from_list):
        Message.__init__(self, filename, loc)
        self.message_args = (name, from_list)


class UndefinedName(Message):
    message = 'undefined name %r'

    def __init__(self, filename, loc, name):
        Message.__init__(self, filename, loc)
        self.message_args = (name,)


class DoctestSyntaxError(Message):
    message = 'syntax error in doctest'

    def __init__(self, filename, loc, position=None):
        Message.__init__(self, filename, loc)
        if position:
            (self.lineno, self.col) = position
        self.message_args = ()


class UndefinedExport(Message):
    message = 'undefined name %r in __all__'

    def __init__(self, filename, loc, name):
        Message.__init__(self, filename, loc)
        self.message_args = (name,)


class UndefinedLocal(Message):
    message = 'local variable %r {0} referenced before assignment'

    default = 'defined in enclosing scope on line %r'
    builtin = 'defined as a builtin'

    def __init__(self, filename, loc, name, orig_loc):
        Message.__init__(self, filename, loc)
        if orig_loc is None:
            self.message = self.message.format(self.builtin)
            self.message_args = name
        else:
            self.message = self.message.format(self.default)
            self.message_args = (name, orig_loc.lineno)


class DuplicateArgument(Message):
    message = 'duplicate argument %r in function definition'

    def __init__(self, filename, loc, name):
        Message.__init__(self, filename, loc)
        self.message_args = (name,)


class MultiValueRepeatedKeyLiteral(Message):
    message = 'dictionary key %r repeated with different values'

    def __init__(self, filename, loc, key):
        Message.__init__(self, filename, loc)
        self.message_args = (key,)


class MultiValueRepeatedKeyVariable(Message):
    message = 'dictionary key variable %s repeated with different values'

    def __init__(self, filename, loc, key):
        Message.__init__(self, filename, loc)
        self.message_args = (key,)


class LateFutureImport(Message):
    message = 'from __future__ imports must occur at the beginning of the file'


class FutureFeatureNotDefined(Message):
    """An undefined __future__ feature name was imported."""
    message = 'future feature %s is not defined'

    def __init__(self, filename, loc, name):
        Message.__init__(self, filename, loc)
        self.message_args = (name,)


class UnusedVariable(Message):
    """
    Indicates that a variable has been explicitly assigned to but not actually
    used.
    """
    message = 'local variable %r is assigned to but never used'

    def __init__(self, filename, loc, names):
        Message.__init__(self, filename, loc)
        self.message_args = (names,)


class UnusedAnnotation(Message):
    """
    Indicates that a variable has been explicitly annotated to but not actually
    used.
    """
    message = 'local variable %r is annotated but never used'

    def __init__(self, filename, loc, names):
        Message.__init__(self, filename, loc)
        self.message_args = (names,)


class UnusedIndirectAssignment(Message):
    """A `global` or `nonlocal` statement where the name is never reassigned"""
    message = '`%s %s` is unused: name is never assigned in scope'

    def __init__(self, filename, loc, name):
        Message.__init__(self, filename, loc)
        self.message_args = (type(loc).__name__.lower(), name)


class ReturnOutsideFunction(Message):
    """
    Indicates a return statement outside of a function/method.
    """
    message = '\'return\' outside function'


class YieldOutsideFunction(Message):
    """
    Indicates a yield or yield from statement outside of a function/method.
    """
    message = '\'yield\' outside function'


# For whatever reason, Python gives different error messages for these two. We
# match the Python error message exactly.
class ContinueOutsideLoop(Message):
    """
    Indicates a continue statement outside of a while or for loop.
    """
    message = '\'continue\' not properly in loop'


class BreakOutsideLoop(Message):
    """
    Indicates a break statement outside of a while or for loop.
    """
    message = '\'break\' outside loop'


class DefaultExceptNotLast(Message):
    """
    Indicates an except: block as not the last exception handler.
    """
    message = 'default \'except:\' must be last'


class TwoStarredExpressions(Message):
    """
    Two or more starred expressions in an assignment (a, *b, *c = d).
    """
    message = 'two starred expressions in assignment'


class TooManyExpressionsInStarredAssignment(Message):
    """
    Too many expressions in an assignment with star-unpacking
    """
    message = 'too many expressions in star-unpacking assignment'


class IfTuple(Message):
    """
    Conditional test is a non-empty tuple literal, which are always True.
    """
    message = '\'if tuple literal\' is always true, perhaps remove accidental comma?'


class AssertTuple(Message):
    """
    Assertion test is a non-empty tuple literal, which are always True.
    """
    message = 'assertion is always true, perhaps remove parentheses?'


class ForwardAnnotationSyntaxError(Message):
    message = 'syntax error in forward annotation %r'

    def __init__(self, filename, loc, annotation):
        Message.__init__(self, filename, loc)
        self.message_args = (annotation,)


class RaiseNotImplemented(Message):
    message = "'raise NotImplemented' should be 'raise NotImplementedError'"


class InvalidPrintSyntax(Message):
    message = 'use of >> is invalid with print function'


class IsLiteral(Message):
    message = 'use ==/!= to compare constant literals (str, bytes, int, float, tuple)'


class FStringMissingPlaceholders(Message):
    message = 'f-string is missing placeholders'


class TStringMissingPlaceholders(Message):
    message = 't-string is missing placeholders'


class StringDotFormatExtraPositionalArguments(Message):
    message = "'...'.format(...) has unused arguments at position(s): %s"

    def __init__(self, filename, loc, extra_positions):
        Message.__init__(self, filename, loc)
        self.message_args = (extra_positions,)


class StringDotFormatExtraNamedArguments(Message):
    message = "'...'.format(...) has unused named argument(s): %s"

    def __init__(self, filename, loc, extra_keywords):
        Message.__init__(self, filename, loc)
        self.message_args = (extra_keywords,)


class StringDotFormatMissingArgument(Message):
    message = "'...'.format(...) is missing argument(s) for placeholder(s): %s"

    def __init__(self, filename, loc, missing_arguments):
        Message.__init__(self, filename, loc)
        self.message_args = (missing_arguments,)


class StringDotFormatMixingAutomatic(Message):
    message = "'...'.format(...) mixes automatic and manual numbering"


class StringDotFormatInvalidFormat(Message):
    message = "'...'.format(...) has invalid format string: %s"

    def __init__(self, filename, loc, error):
        Message.__init__(self, filename, loc)
        self.message_args = (error,)


class PercentFormatInvalidFormat(Message):
    message = "'...' %% ... has invalid format string: %s"

    def __init__(self, filename, loc, error):
        Message.__init__(self, filename, loc)
        self.message_args = (error,)


class PercentFormatMixedPositionalAndNamed(Message):
    message = "'...' %% ... has mixed positional and named placeholders"


class PercentFormatUnsupportedFormatCharacter(Message):
    message = "'...' %% ... has unsupported format character %r"

    def __init__(self, filename, loc, c):
        Message.__init__(self, filename, loc)
        self.message_args = (c,)


class PercentFormatPositionalCountMismatch(Message):
    message = "'...' %% ... has %d placeholder(s) but %d substitution(s)"

    def __init__(self, filename, loc, n_placeholders, n_substitutions):
        Message.__init__(self, filename, loc)
        self.message_args = (n_placeholders, n_substitutions)


class PercentFormatExtraNamedArguments(Message):
    message = "'...' %% ... has unused named argument(s): %s"

    def __init__(self, filename, loc, extra_keywords):
        Message.__init__(self, filename, loc)
        self.message_args = (extra_keywords,)


class PercentFormatMissingArgument(Message):
    message = "'...' %% ... is missing argument(s) for placeholder(s): %s"

    def __init__(self, filename, loc, missing_arguments):
        Message.__init__(self, filename, loc)
        self.message_args = (missing_arguments,)


class PercentFormatExpectedMapping(Message):
    message = "'...' %% ... expected mapping but got sequence"


class PercentFormatExpectedSequence(Message):
    message = "'...' %% ... expected sequence but got mapping"


class PercentFormatStarRequiresSequence(Message):
    message = "'...' %% ... `*` specifier requires sequence"
```

**File:** `~/.local/lib/python3.12/site-packages/pyflakes/messages.py`

In [ ]:
pfmod = pyflakes.messages
syms = [_ for _ in dir(pfmod) if _[0] != '_']
    

In [ ]:
kls_msg = pfmod.Message
kls = getattr(pfmod, 'UnusedVariable')
kls_msg in kls.mro()

True

In [ ]:
[sym for sym in syms if isinstance(o := getattr(pfmod, sym), type) and kls_msg in o.mro()]
        

['AssertTuple',
 'BreakOutsideLoop',
 'ContinueOutsideLoop',
 'DefaultExceptNotLast',
 'DoctestSyntaxError',
 'DuplicateArgument',
 'FStringMissingPlaceholders',
 'ForwardAnnotationSyntaxError',
 'FutureFeatureNotDefined',
 'IfTuple',
 'ImportShadowedByLoopVar',
 'ImportStarNotPermitted',
 'ImportStarUsage',
 'ImportStarUsed',
 'InvalidPrintSyntax',
 'IsLiteral',
 'LateFutureImport',
 'Message',
 'MultiValueRepeatedKeyLiteral',
 'MultiValueRepeatedKeyVariable',
 'PercentFormatExpectedMapping',
 'PercentFormatExpectedSequence',
 'PercentFormatExtraNamedArguments',
 'PercentFormatInvalidFormat',
 'PercentFormatMissingArgument',
 'PercentFormatMixedPositionalAndNamed',
 'PercentFormatPositionalCountMismatch',
 'PercentFormatStarRequiresSequence',
 'PercentFormatUnsupportedFormatCharacter',
 'RaiseNotImplemented',
 'RedefinedWhileUnused',
 'ReturnOutsideFunction',
 'StringDotFormatExtraNamedArguments',
 'StringDotFormatExtraPositionalArguments',
 'StringDotFormatInvalidFormat',
 'StringDot

In [ ]:
pfmod.UnusedVariable.__doc__

'\n    Indicates that a variable has been explicitly assigned to but not actually\n    used.\n    '

In [ ]:
#| export
def _warningtypes():
    "Return all pyflakes message types"
    pfmod = pyflakes.messages
    kls_msg = pfmod.Message
    syms = [_ for _ in dir(pfmod) if _[0] != '_']
    return {sym: o.__doc__.strip() if o.__doc__ else o.message
        for sym in syms 
        if sym != 'Message' and isinstance(o := getattr(pfmod, sym), type) and kls_msg in o.mro()}

In [ ]:
#| export
_ALL = ','.join(_warningtypes().keys())
_IMPORTS = 'ImportShadowedByLoopVar,ImportStarNotPermitted,ImportStarUsage,ImportStarUsed,LateFutureImport,UnusedImport'
_VARS = 'UndefinedExport,UndefinedLocal,UndefinedName,UnusedIndirectAssignment,UnusedVariable'


In [ ]:
_ALL

'AssertTuple,BreakOutsideLoop,ContinueOutsideLoop,DefaultExceptNotLast,DoctestSyntaxError,DuplicateArgument,FStringMissingPlaceholders,ForwardAnnotationSyntaxError,FutureFeatureNotDefined,IfTuple,ImportShadowedByLoopVar,ImportStarNotPermitted,ImportStarUsage,ImportStarUsed,InvalidPrintSyntax,IsLiteral,LateFutureImport,MultiValueRepeatedKeyLiteral,MultiValueRepeatedKeyVariable,PercentFormatExpectedMapping,PercentFormatExpectedSequence,PercentFormatExtraNamedArguments,PercentFormatInvalidFormat,PercentFormatMissingArgument,PercentFormatMixedPositionalAndNamed,PercentFormatPositionalCountMismatch,PercentFormatStarRequiresSequence,PercentFormatUnsupportedFormatCharacter,RaiseNotImplemented,RedefinedWhileUnused,ReturnOutsideFunction,StringDotFormatExtraNamedArguments,StringDotFormatExtraPositionalArguments,StringDotFormatInvalidFormat,StringDotFormatMissingArgument,StringDotFormatMixingAutomatic,TStringMissingPlaceholders,TooManyExpressionsInStarredAssignment,TwoStarredExpressions,Undefined

## v3

In [ ]:
def check_flakes(dname:str=''):
    "Check for pyflakes 'warnings' in `sname` or current dialog"
    cds = find_msgs(msg_type='code', dname=dname)
    exptd = cds.filter(lambda x: x.is_exported)
    src, l2id = _get_source(exptd)
    nw = _check(src, Path(dname or find_dname()).name, reprt := ListReporter())
    return nw, l2id, reprt.warnings
    

In [ ]:
nw, l2id, wrns = check_flakes()
print(nw)
for _ in wrns[:5]: print(_)

23
('UndefinedName', '05_flakes', (2, 2), "undefined name 'FC'", ('FC',))
('UndefinedName', '05_flakes', (3, 20), "undefined name 'Message'", ('Message',))
('UndefinedName', '05_flakes', (5, 20), "undefined name 'Reporter'", ('Reporter',))
('UndefinedName', '05_flakes', (6, 41), "undefined name 'FC'", ('FC',))
('UndefinedName', '05_flakes', (10, 21), "undefined name 'FC'", ('FC',))


In [ ]:
nw, l2id, wrns = check_flakes(dname=_DREL)
print(nw)
for _ in wrns[:5]: print(_)

7
('UnusedImport', '00_core', (6, 1), "'fastcore.all as FC' imported but unused", ('fastcore.all as FC',))
('ImportStarUsed', '00_core', (9, 1), "'from dialoghelper import *' used; unable to detect undefined names", ('dialoghelper',))
('ImportStarUsage', '00_core', (62, 20), "'is_usable_tool' may be undefined, or defined from star imports: dialoghelper", ('is_usable_tool', 'dialoghelper'))
('ImportStarUsage', '00_core', (68, 5), "'add_msg' may be undefined, or defined from star imports: dialoghelper", ('add_msg', 'dialoghelper'))
('ImportStarUsage', '00_core', (68, 13), "'mk_toollist' may be undefined, or defined from star imports: dialoghelper", ('mk_toollist', 'dialoghelper'))


## filter

In [ ]:
#| export
def check_flakes(dname:str='', wtypes:str=_ALL):
    "Check for pyflakes 'warnings' in `sname` or current dialog"
    cds = find_msgs(msg_type='code', dname=dname)
    src, l2id = _get_source(cds.filter(lambda x: x.is_exported))
    nw = _check(src, Path(dname or find_dname()).name, reprt := ListReporter())
    if wtypes != _ALL: wtypes = wtypes.split(',')
    return nw, l2id, reprt.warnings if wtypes == _ALL else reprt.warnings.filter(lambda x: x[0] in wtypes)

In [ ]:
n, l2id, wns = check_flakes(_DREL, wtypes='UnusedImport')
print(wns)

[('UnusedImport', '00_core', (6, 1), "'fastcore.all as FC' imported but unused", ('fastcore.all as FC',))]


In [ ]:
n, l2id, wns = check_flakes(_DABS)
for wn in wns: print(wn)

('UnusedImport', '00_core', (6, 1), "'fastcore.all as FC' imported but unused", ('fastcore.all as FC',))
('ImportStarUsed', '00_core', (9, 1), "'from dialoghelper import *' used; unable to detect undefined names", ('dialoghelper',))
('ImportStarUsage', '00_core', (62, 20), "'is_usable_tool' may be undefined, or defined from star imports: dialoghelper", ('is_usable_tool', 'dialoghelper'))
('ImportStarUsage', '00_core', (68, 5), "'add_msg' may be undefined, or defined from star imports: dialoghelper", ('add_msg', 'dialoghelper'))
('ImportStarUsage', '00_core', (68, 13), "'mk_toollist' may be undefined, or defined from star imports: dialoghelper", ('mk_toollist', 'dialoghelper'))
('ImportStarUsage', '00_core', (74, 14), "'find_msg_id' may be undefined, or defined from star imports: dialoghelper", ('find_msg_id', 'dialoghelper'))
('ImportStarUsage', '00_core', (76, 13), "'add_msg' may be undefined, or defined from star imports: dialoghelper", ('add_msg', 'dialoghelper'))


In [ ]:
n, l2id, wns = check_flakes()
for wn in wns: print(wn)


('UndefinedName', '05_flakes', (2, 2), "undefined name 'FC'", ('FC',))
('UndefinedName', '05_flakes', (3, 20), "undefined name 'Message'", ('Message',))
('UndefinedName', '05_flakes', (5, 20), "undefined name 'Reporter'", ('Reporter',))
('UndefinedName', '05_flakes', (6, 41), "undefined name 'FC'", ('FC',))
('UndefinedName', '05_flakes', (10, 21), "undefined name 'FC'", ('FC',))
('UndefinedName', '05_flakes', (14, 54), "undefined name 'repeat'", ('repeat',))
('UndefinedName', '05_flakes', (24, 13), "undefined name 'pyflakes'", ('pyflakes',))
('UndefinedName', '05_flakes', (38, 11), "undefined name 'find_msgs'", ('find_msgs',))
('UndefinedName', '05_flakes', (40, 10), "undefined name '_check'", ('_check',))
('UndefinedName', '05_flakes', (40, 22), "undefined name 'Path'", ('Path',))
('UndefinedName', '05_flakes', (40, 36), "undefined name 'find_dname'", ('find_dname',))
('UndefinedName', '05_flakes', (46, 11), "undefined name 'defaultdict'", ('defaultdict',))
('UndefinedName', '05_flake

## pretty print

In [ ]:
#| export
def _group_flakes(wns, l2id):
    "Group pyflakes warnings by msgid and warning type"
    res = defaultdict(lambda:defaultdict(list))
    for wtype, _, rng, *rest in wns:
        msgid = l2id[rng[0]][2]
        res[msgid][wtype].append((rng, rest))
    return res

In [ ]:
n, l2id, wns = check_flakes(_DREL)

grouped = _group_flakes(wns, l2id)
grouped

defaultdict(<function __main__._group_flakes.<locals>.<lambda>()>,
            {'_e72f67fd': defaultdict(list,
                         {'UnusedImport': [((6, 1),
                            ["'fastcore.all as FC' imported but unused",
                             ('fastcore.all as FC',)])],
                          'ImportStarUsed': [((9, 1),
                            ["'from dialoghelper import *' used; unable to detect undefined names",
                             ('dialoghelper',)])]}),
             '_9027bb28': defaultdict(list,
                         {'ImportStarUsage': [((62, 20),
                            ["'is_usable_tool' may be undefined, or defined from star imports: dialoghelper",
                             ('is_usable_tool', 'dialoghelper')])]}),
             '_6a2bafc4': defaultdict(list,
                         {'ImportStarUsage': [((68, 5),
                            ["'add_msg' may be undefined, or defined from star imports: dialoghelper",
                

In [ ]:
#| export
def _tag(name, msgid=''): return f"<!-- {name}: {msgid or find_msg_id()} -->"

In [ ]:
#| export
_lnks = (
    '''<span hx-on-click="setTimeout(() => selectMsg($('%s'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**%s**</span>  \n\n''',
    '''<h5 class="uk-flex"><a class="uk-link" href="%s"><strong>%s</strong></svg></a>&nbsp;<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" height="16px" width="16px" class="lucide-icon "><use href="#lc-external-link"></use></h5>  \n\n'''
)

In [ ]:
s = f"{_tag('flakes report')}\n"
for msgid, wtypes in grouped.items():
    s += _lnks[1] % (msgid, msgid)#.format(msgid, msgid)
    for wtype, ws in wtypes.items():
        s += f"- **{wtype}**: {len(ws)}  \n"
        for w in ws: s += f"    `{w[0]}`: {w[1][0].replace("'", "`")}  \n"
    s += '\n'
print(s[:200])

<!-- flakes report: _0b0fae16 -->
<h5 class="uk-flex"><a class="uk-link" href="_e72f67fd"><strong>_e72f67fd</strong></svg></a>&nbsp;<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" height="


In [ ]:
dname = _DREL
rel = not dname or dname[0] != '/'
str(Path(find_dname()).parent/dname) if rel else dname[1:]

'vic/dutil/nbs/00_core'

In [ ]:
dname = _DABS
rel = not dname or dname[0] != '/'
str(Path(find_dname()).parent/dname) if rel else dname[1:]

'vic/dutil/nbs/00_core'

In [ ]:
#| export
def _render_flakes_report(dname, wns, l2id):
    tag = _tag('flakes report')
    dlnk, s = '', f"{tag}\n"
    if dname:
        q = quote_plus(str(Path(find_dname()).parent/dname) if (not dname or dname[0] != '/') else dname[1:])
        dlnk = f"/dialog_?name={q}"
    grouped = _group_flakes(wns, l2id)
    for msgid, wtypes in grouped.items():
        lnk = f"{dlnk}#{msgid}"
        s += _lnks[bool(dname)] % (lnk, msgid)
        for wtype, ws in wtypes.items():
            s += f"- **{wtype}**: {len(ws)}  \n"
            for w in ws: s += f"    `{w[0]}`: {w[1][0].replace("'", "`")}  \n"
        s += '\n'
    return s, tag

In [ ]:
n, l2id, wns = check_flakes(_DREL)
s, tag = _render_flakes_report(_DREL, wns, l2id)
print(s[:200])

<!-- flakes report: _af461bb3 -->
<h5 class="uk-flex"><a class="uk-link" href="/dialog_?name=vic%2Fdutil%2Fnbs%2F00_core#_e72f67fd"><strong>_e72f67fd</strong></svg></a>&nbsp;<svg xmlns="http://www.w3.


In [ ]:
n, l2id, wns = check_flakes(_DABS)
s, tag = _render_flakes_report(_DABS, wns, l2id)
print(s[:200])

<!-- flakes report: _a534cf17 -->
<h5 class="uk-flex"><a class="uk-link" href="/dialog_?name=vic%2Fdutil%2Fnbs%2F00_core#_e72f67fd"><strong>_e72f67fd</strong></svg></a>&nbsp;<svg xmlns="http://www.w3.


In [ ]:
n, l2id, wns = check_flakes()
s, tag = _render_flakes_report('', wns, l2id)
print(s[:200])

<!-- flakes report: _a6789212 -->
<span hx-on-click="setTimeout(() => selectMsg($('#_42c1d335'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_42


In [ ]:
cprint(s)

<!-- flakes report: _a6789212 -->
<span hx-on-click="setTimeout(() => selectMsg($('#_42c1d335'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer 
hover:bg-muted truncate">**_42c1d335**</span>  

- **UndefinedName**: 4  
    `(2, 2)`: undefined name `FC`  
    `(3, 20)`: undefined name `Message`  
    `(5, 20)`: undefined name `Reporter`  
    `(6, 41)`: undefined name `FC`  

<span hx-on-click="setTimeout(() => selectMsg($('#_28266a58'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer 
hover:bg-muted truncate">**_28266a58**</span>  

- **UndefinedName**: 2  
    `(10, 21)`: undefined name `FC`  
    `(14, 54)`: undefined name `repeat`  

<span hx-on-click="setTimeout(() => selectMsg($('#_5e564084'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer 
hover:bg-muted truncate">**_5e564084**</span>  

- **UndefinedName**: 1  
    `(24, 13)`: undefined name `pyflakes`  

<span hx-on-click="setTimeout(() => selectMsg($('#_65ee988a'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer 
hover:bg-muted truncate">**_65ee988a**</span>  

- **UndefinedName**: 4  
    `(38, 11)`: undefined name `find_msgs`  
    `(40, 10)`: undefined name `_check`  
    `(40, 22)`: undefined name `Path`  
    `(40, 36)`: undefined name `find_dname`  

<span hx-on-click="setTimeout(() => selectMsg($('#_252cd459'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer 
hover:bg-muted truncate">**_252cd459**</span>  

- **UndefinedName**: 2  
    `(46, 11)`: undefined name `defaultdict`  
    `(46, 30)`: undefined name `defaultdict`  

<span hx-on-click="setTimeout(() => selectMsg($('#_47ffa603'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer 
hover:bg-muted truncate">**_47ffa603**</span>  

- **UndefinedName**: 1  
    `(52, 59)`: undefined name `find_msg_id`  

<span hx-on-click="setTimeout(() => selectMsg($('#_9513dbdc'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer 
hover:bg-muted truncate">**_9513dbdc**</span>  

- **UndefinedName**: 3  
    `(63, 13)`: undefined name `quote_plus`  
    `(63, 28)`: undefined name `Path`  
    `(63, 33)`: undefined name `find_dname`  

<span hx-on-click="setTimeout(() => selectMsg($('#_5084f06f'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer 
hover:bg-muted truncate">**_5084f06f**</span>  

- **UndefinedName**: 6  
    `(78, 22)`: undefined name `find_msg_id`  
    `(79, 14)`: undefined name `find_var`  
    `(81, 15)`: undefined name `read_msg`  
    `(82, 33)`: undefined name `re`  
    `(83, 29)`: undefined name `update_msg`  
    `(85, 21)`: undefined name `add_msg`

## report gen

In [ ]:
#| export
get_ipython().xpush(__linked_msgs={})  # WARNING: neither get_ipython (in user_ns) nor ipykernel_helper.xpush are documented

In [ ]:
'__linked_msgs' in get_ipython().user_ns_hidden

True

In [ ]:
__linked_msgs

{}

In [ ]:
linked = find_var('__linked_msgs')
linked

{}

In [ ]:
if reportid := linked.get(thisid := find_msg_id()):
    msg = read_msg(0, True, reportid, nums=True)
    if 'msg' not in msg and re.search(tag, msg.content):
        linked[thisid] = update_msg(reportid, content=s)
    else: print('not found')
else:
    linked[thisid] = add_msg(s)

<!-- flakes report: _a6789212 -->
<span hx-on-click="setTimeout(() => selectMsg($('#_42c1d335'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_42c1d335**</span>  

- **UndefinedName**: 4  
    `(2, 2)`: undefined name `FC`  
    `(3, 20)`: undefined name `Message`  
    `(5, 20)`: undefined name `Reporter`  
    `(6, 41)`: undefined name `FC`  

<span hx-on-click="setTimeout(() => selectMsg($('#_28266a58'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_28266a58**</span>  

- **UndefinedName**: 2  
    `(10, 21)`: undefined name `FC`  
    `(14, 54)`: undefined name `repeat`  

<span hx-on-click="setTimeout(() => selectMsg($('#_5e564084'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_5e564084**</span>  

- **UndefinedName**: 1  
    `(24, 13)`: undefined name `pyflakes`  

<span hx-on-click="setTimeout(() => selectMsg($('#_65ee988a'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_65ee988a**</span>  

- **UndefinedName**: 4  
    `(38, 11)`: undefined name `find_msgs`  
    `(40, 10)`: undefined name `_check`  
    `(40, 22)`: undefined name `Path`  
    `(40, 36)`: undefined name `find_dname`  

<span hx-on-click="setTimeout(() => selectMsg($('#_252cd459'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_252cd459**</span>  

- **UndefinedName**: 2  
    `(46, 11)`: undefined name `defaultdict`  
    `(46, 30)`: undefined name `defaultdict`  

<span hx-on-click="setTimeout(() => selectMsg($('#_47ffa603'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_47ffa603**</span>  

- **UndefinedName**: 1  
    `(52, 59)`: undefined name `find_msg_id`  

<span hx-on-click="setTimeout(() => selectMsg($('#_9513dbdc'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_9513dbdc**</span>  

- **UndefinedName**: 3  
    `(63, 13)`: undefined name `quote_plus`  
    `(63, 28)`: undefined name `Path`  
    `(63, 33)`: undefined name `find_dname`  

<span hx-on-click="setTimeout(() => selectMsg($('#_5084f06f'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_5084f06f**</span>  

- **UndefinedName**: 6  
    `(78, 22)`: undefined name `find_msg_id`  
    `(79, 14)`: undefined name `find_var`  
    `(81, 15)`: undefined name `read_msg`  
    `(82, 33)`: undefined name `re`  
    `(83, 29)`: undefined name `update_msg`  
    `(85, 21)`: undefined name `add_msg`  



In [ ]:
#| export
def _update_linked_msg(content, tag, msgid=''):
    msgid = msgid or find_msg_id()
    linked = find_var('__linked_msgs')
    if reportid := linked.get(msgid):
        msg = read_msg(0, True, reportid, nums=True)
        if 'msg' not in msg and re.search(tag, msg.content):
            linked[msgid] = update_msg(reportid, content=content)
            return
    linked[msgid] = add_msg(content)

In [ ]:
thisid = find_msg_id()
_update_linked_msg(s, tag, thisid)

<!-- flakes report: _a6789212 -->
<span hx-on-click="setTimeout(() => selectMsg($('#_42c1d335'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_42c1d335**</span>  

- **UndefinedName**: 4  
    `(2, 2)`: undefined name `FC`  
    `(3, 20)`: undefined name `Message`  
    `(5, 20)`: undefined name `Reporter`  
    `(6, 41)`: undefined name `FC`  

<span hx-on-click="setTimeout(() => selectMsg($('#_28266a58'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_28266a58**</span>  

- **UndefinedName**: 2  
    `(10, 21)`: undefined name `FC`  
    `(14, 54)`: undefined name `repeat`  

<span hx-on-click="setTimeout(() => selectMsg($('#_5e564084'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_5e564084**</span>  

- **UndefinedName**: 1  
    `(24, 13)`: undefined name `pyflakes`  

<span hx-on-click="setTimeout(() => selectMsg($('#_65ee988a'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_65ee988a**</span>  

- **UndefinedName**: 4  
    `(38, 11)`: undefined name `find_msgs`  
    `(40, 10)`: undefined name `_check`  
    `(40, 22)`: undefined name `Path`  
    `(40, 36)`: undefined name `find_dname`  

<span hx-on-click="setTimeout(() => selectMsg($('#_252cd459'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_252cd459**</span>  

- **UndefinedName**: 2  
    `(46, 11)`: undefined name `defaultdict`  
    `(46, 30)`: undefined name `defaultdict`  

<span hx-on-click="setTimeout(() => selectMsg($('#_47ffa603'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_47ffa603**</span>  

- **UndefinedName**: 1  
    `(52, 59)`: undefined name `find_msg_id`  

<span hx-on-click="setTimeout(() => selectMsg($('#_9513dbdc'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_9513dbdc**</span>  

- **UndefinedName**: 3  
    `(63, 13)`: undefined name `quote_plus`  
    `(63, 28)`: undefined name `Path`  
    `(63, 33)`: undefined name `find_dname`  

<span hx-on-click="setTimeout(() => selectMsg($('#_5084f06f'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_5084f06f**</span>  

- **UndefinedName**: 6  
    `(78, 22)`: undefined name `find_msg_id`  
    `(79, 14)`: undefined name `find_var`  
    `(81, 15)`: undefined name `read_msg`  
    `(82, 33)`: undefined name `re`  
    `(83, 29)`: undefined name `update_msg`  
    `(85, 21)`: undefined name `add_msg`  



In [ ]:
find_var('__linked_msgs')

{'_d730d208': '_3ad0494d', '_b2d48814': '_5ab0222c'}

**NOTE**: that could be a useful pattern, linked messages -> see exploration [linked_msgs & msg custom metadata](/dialog_?name=vic/explorer/linked_msg)

## all together now


In [ ]:
#| export
def add_flakes(
    dname:str='',  # Dialog to check; defaults to current dialog
    wtypes:str=_ALL # comma separated list of warning types to include; defaults to all
):
    """Add a message below with the warnings generated by pyflakes.
If `dname` is None, the current dialog is used. Dialog names other than None must be paths relative to the solveit root directory (if starting with `/`) or relative to the current dialog (if not starting with `/`), and should *not* include the .ipynb extension."""
    n, l2id, wns = check_flakes(dname, wtypes)
    s, tag = _render_flakes_report(dname, wns, l2id)
    _update_linked_msg(s, tag)

In [ ]:
add_flakes(_DREL)

<!-- flakes report: _114c5311 -->
<h5 class="uk-flex"><a class="uk-link" href="/dialog_?name=vic%2Fdutil%2Fnbs%2F00_core#_e72f67fd"><strong>_e72f67fd</strong></svg></a>&nbsp;<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" height="16px" width="16px" class="lucide-icon "><use href="#lc-external-link"></use></h5>  

- **UnusedImport**: 1  
    `(6, 1)`: `fastcore.all as FC` imported but unused  
- **ImportStarUsed**: 1  
    `(9, 1)`: `from dialoghelper import *` used; unable to detect undefined names  

<h5 class="uk-flex"><a class="uk-link" href="/dialog_?name=vic%2Fdutil%2Fnbs%2F00_core#_9027bb28"><strong>_9027bb28</strong></svg></a>&nbsp;<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" height="16px" width="16px" class="lucide-icon "><use href="#lc-external-link"></use></h5>  

- **ImportStarUsage**: 1  
    `(62, 20)`: `is_usable_tool` may be undefined, or defined from star imports: dialoghelper  

<h5 class="uk-flex"><a class="uk-link" href="/dialog_?name=vic%2Fdutil%2Fnbs%2F00_core#_6a2bafc4"><strong>_6a2bafc4</strong></svg></a>&nbsp;<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 24 24" height="16px" width="16px" class="lucide-icon "><use href="#lc-external-link"></use></h5>  

- **ImportStarUsage**: 4  
    `(68, 5)`: `add_msg` may be undefined, or defined from star imports: dialoghelper  
    `(68, 13)`: `mk_toollist` may be undefined, or defined from star imports: dialoghelper  
    `(74, 14)`: `find_msg_id` may be undefined, or defined from star imports: dialoghelper  
    `(76, 13)`: `add_msg` may be undefined, or defined from star imports: dialoghelper  



The first imports block at the beginning of the dialog is not exported. So `check_flakes` reports many `UndefinedName':

In [ ]:
add_flakes()

<!-- flakes report: _60bcadc0 -->
<span hx-on-click="setTimeout(() => selectMsg($('#_42c1d335'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_42c1d335**</span>  

- **UndefinedName**: 4  
    `(2, 2)`: undefined name `FC`  
    `(3, 20)`: undefined name `Message`  
    `(5, 20)`: undefined name `Reporter`  
    `(6, 41)`: undefined name `FC`  

<span hx-on-click="setTimeout(() => selectMsg($('#_28266a58'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_28266a58**</span>  

- **UndefinedName**: 2  
    `(10, 21)`: undefined name `FC`  
    `(14, 54)`: undefined name `repeat`  

<span hx-on-click="setTimeout(() => selectMsg($('#_5e564084'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_5e564084**</span>  

- **UndefinedName**: 1  
    `(24, 13)`: undefined name `pyflakes`  

<span hx-on-click="setTimeout(() => selectMsg($('#_65ee988a'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_65ee988a**</span>  

- **UndefinedName**: 4  
    `(38, 11)`: undefined name `find_msgs`  
    `(40, 10)`: undefined name `_check`  
    `(40, 22)`: undefined name `Path`  
    `(40, 36)`: undefined name `find_dname`  

<span hx-on-click="setTimeout(() => selectMsg($('#_252cd459'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_252cd459**</span>  

- **UndefinedName**: 2  
    `(46, 11)`: undefined name `defaultdict`  
    `(46, 30)`: undefined name `defaultdict`  

<span hx-on-click="setTimeout(() => selectMsg($('#_47ffa603'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_47ffa603**</span>  

- **UndefinedName**: 1  
    `(52, 59)`: undefined name `find_msg_id`  

<span hx-on-click="setTimeout(() => selectMsg($('#_9513dbdc'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_9513dbdc**</span>  

- **UndefinedName**: 3  
    `(63, 13)`: undefined name `quote_plus`  
    `(63, 28)`: undefined name `Path`  
    `(63, 33)`: undefined name `find_dname`  

<span hx-on-click="setTimeout(() => selectMsg($('#_5084f06f'), {centered: true}), 100)" class="uk-link text-blue-600 p-1 cursor-pointer hover:bg-muted truncate">**_5084f06f**</span>  

- **UndefinedName**: 6  
    `(78, 22)`: undefined name `find_msg_id`  
    `(79, 14)`: undefined name `find_var`  
    `(81, 15)`: undefined name `read_msg`  
    `(82, 33)`: undefined name `re`  
    `(83, 29)`: undefined name `update_msg`  
    `(85, 21)`: undefined name `add_msg`  



Go now to the first import block and mark it for export. Come back here and run `add_flakes` below.

# export -

In [ ]:
add_flakes()

<!-- flakes report: _b28df02e -->


In [ ]:
#|hide
#|eval: false
import fastcore.all as FC
from nbdev import nbdev_export
if FC.IN_NOTEBOOK:
    nb_path = '05_flakes.ipynb'
    nbdev_export(nb_path)